In [2]:
import pandas as pd
import numpy as np

from lightgbm import LGBMClassifier
from sklearn.preprocessing import OneHotEncoder
import sys
sys.path.append("/home/leon/projects/gaussian_process/")
from sklearn.model_selection import cross_val_score
from gecs100.gec import GEC
from gecs100.utils.gaussian_process_visualisation import visualise_1D_gaussian_process, visualise_2D_gaussian_process

gec = GEC()
#gec.load_gp_datas("./gp_datas.json")


In [3]:
path = "/home/leon/learning/gaussian-processes/bank/bank-full.csv"
data = pd.read_csv(path, sep=";")
one_hot_encode = ["job", "marital", "education", "contact", "poutcome", "month"]
binary = ["default", "housing", "loan", "y"]

enc = OneHotEncoder()
def yesNoBinary(column):
    return(pd.DataFrame([1 if value == "yes" else 0 for value in column], columns = [column.name]))
one_hot = pd.DataFrame(enc.fit_transform(data[one_hot_encode]).toarray(), columns=enc.get_feature_names_out(one_hot_encode))
data2 = pd.concat([one_hot] + [ yesNoBinary(data[col]) if col in binary else data[col] for col in data.columns ], 1).drop(one_hot_encode, 1)
X, y = data2.values[:,:-1], data2.values[:,-1]
np.random.seed(101)
ind = np.random.uniform(0, 1, X.shape[0]) <1.1
X = X[ind,:]
y = y[ind]

X_pos = X[y==1,:]
y_pos = y[y==1]

X_reweighted = np.concatenate([X] + [X_pos]*3, axis=0)
y_reweighted = np.concatenate([y] + [y_pos]*3)


print(X_reweighted.shape)

(61078, 48)


/tmp/ipykernel_98479/1665828294.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  data2 = pd.concat([one_hot] + [ yesNoBinary(data[col]) if col in binary else data[col] for col in data.columns ], 1).drop(one_hot_encode, 1)
/tmp/ipykernel_98479/1665828294.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data2 = pd.concat([one_hot] + [ yesNoBinary(data[col]) if col in binary else data[col] for col in data.columns ], 1).drop(one_hot_encode, 1)


In [4]:
gec.fit(X_reweighted, y_reweighted, 100)

  2%|▏         | 2/100 [00:07<05:28,  3.35s/it]

In [5]:
gec.best_scores_gec

{'search': 0.8694631322714814,
 'grid': 0.8357196250934392,
 'grid_from_search': 0.8694631322714814}

In [14]:
gec.best_params_gec

{'search': {'boosting': 'gbdt',
  'num_leaves': 111,
  'learning_rate': 0.05695478297678612,
  'n_estimators': 40,
  'max_bin': 10,
  'max_depth': -1,
  'lambda_l1': 0.06997231495784935,
  'lambda_l2': 0.013702670330891934,
  'min_data_in_leaf': 46,
  'feature_fraction': 0.13,
  'bagging_freq': 5,
  'bagging_fraction': 0.9500000000000001,
  'verbosity': -1},
 'grid': {'boosting': 'rf',
  'num_leaves': 105,
  'learning_rate': 0.0462366730725511,
  'n_estimators': 80,
  'max_bin': 160,
  'max_depth': 250,
  'lambda_l1': 0.4818554701340344,
  'lambda_l2': 0.12827052111465376,
  'min_data_in_leaf': 25,
  'feature_fraction': 0.33999999999999986,
  'bagging_freq': 1,
  'bagging_fraction': 0.55},
 'grid_from_search': {'boosting': 'rf',
  'num_leaves': 104,
  'learning_rate': 0.0462366730725511,
  'n_estimators': 80,
  'max_bin': 160,
  'max_depth': 250,
  'lambda_l1': 0.4818554701340344,
  'lambda_l2': 0.12276712716800227,
  'min_data_in_leaf': 26,
  'feature_fraction': 0.34999999999999987,
 

In [5]:
gec.rewards

{'gbdt-yes_bagging': {'a': 1.460203109103189, 'b': 1.5486043753009384},
 'gbdt-no_bagging': {'a': 1.2850113071866223, 'b': 1.4288622798050694},
 'dart-yes_bagging': {'a': 1.4946812436155419, 'b': 1.1846192789333239},
 'dart-no_bagging': {'a': 1.3302033379682117, 'b': 1.539116217690748},
 'rf-yes_bagging': {'a': 1.657485980288401, 'b': 1.5118268726651305}}

In [ ]:
print({k:(np.mean(v["output"]), np.max(v["output"]), len(v["output"])) for k, v in gec.gp_datas.items()})

In [ ]:
#gec.save_figs("./test")

In [10]:
bayesian_params = gec.best_params_gec["grid_from_search"]
print(bayesian_params)
clf_bayes = LGBMClassifier(**bayesian_params)
score_bayes = np.mean(cross_val_score(clf_bayes, X_reweighted, y_reweighted, cv=5))
score_bayes

{'boosting': 'rf', 'num_leaves': 104, 'learning_rate': 0.0462366730725511, 'n_estimators': 80, 'max_bin': 160, 'max_depth': 250, 'lambda_l1': 0.4818554701340344, 'lambda_l2': 0.12276712716800227, 'min_data_in_leaf': 26, 'feature_fraction': 0.34999999999999987, 'bagging_freq': 1, 'bagging_fraction': 0.55}
[LightGBM] [Warning] lambda_l1 is set=0.4818554701340344, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.4818554701340344
[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l2 is set=0.12276712716800227, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.12276712716800227
[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] feature_fraction is set=0.34999999999999987, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.34999999999999987
[LightGBM] [Warning] min_data_in_leaf is set=2

0.6638518536985107

In [11]:
clf_default = LGBMClassifier()
score_default = np.mean(cross_val_score(clf_default, X_reweighted, y_reweighted, cv=5))
score_default

0.6441064187509264

In [ ]:
knn_bayes = LGBMClassifier(**{'boosting': 'rf', 'num_leaves': 199, 'learning_rate': 0.010023052380778996, 'n_estimators': 10, 'max_bin': 140, 'max_depth': 400, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'min_data_in_leaf': 49, 'feature_fraction': 1.0, 'bagging_freq': 1, 'bagging_fraction': 0.05})
score_bayes = np.mean(cross_val_score(knn_bayes, X_reweighted, y_reweighted, cv=5))
score_bayes

In [ ]:
gec.categorical_hyperparameters

In [12]:
from sklearn.model_selection import RandomizedSearchCV
classifier = LGBMClassifier()
hyperparams = dict(gec.categorical_hyperparameters[:1] + gec.real_hyperparameters)
gs = RandomizedSearchCV(classifier, hyperparams, n_iter=5)
gs.fit(X_reweighted, y_reweighted)

[LightGBM] [Warning] lambda_l1 is set=0.07859473860785454, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.07859473860785454
[LightGBM] [Warning] lambda_l2 is set=0.3588047638210554, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3588047638210554
[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] feature_fraction is set=0.5199999999999998, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5199999999999998
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40


[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /__w/1/s/python-package/compile/src/boosting/rf.hpp, line 35 .

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /__w/1/s/python-package/compile/src/boosting/rf.hpp, line 35 .

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /__w/1/s/python-package/compile/src/boosting/rf.hpp, line 35 .

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /__w/1/s/python-package/compile/src/boosting/rf.hpp, line 35 .

[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /__w/1/s/python-package/compile/src/boosting/rf.hpp, line 35 .



[LightGBM] [Warning] lambda_l1 is set=0.4818554701340344, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.4818554701340344
[LightGBM] [Warning] lambda_l2 is set=0.01964577249277761, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01964577249277761
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.48999999999999977, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.48999999999999977
[LightGBM] [Warning] min_data_in_leaf is set=3, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3
[LightGBM] [Warning] lambda_l1 is set=0.4818554701340344, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.4818554701340344
[LightGBM] [Warning] lambda_l2 is set=0.01964577249277761, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01964577249277761
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored.

/home/leon/miniconda3/envs/dev/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
5 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/leon/miniconda3/envs/dev/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/leon/miniconda3/envs/dev/lib/python3.10/site-packages/lightgbm/sklearn.py", line 967, in fit
    super().fit(X, _y, sample_weight=sample_weight, init_score=init_score, eval_set=valid_sets,
  File "/home/leon/miniconda3/envs/dev/lib/python3.10/site-packages/lightgb

[LightGBM] [Warning] lambda_l1 is set=0.11213700028500526, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.11213700028500526
[LightGBM] [Warning] lambda_l2 is set=0.04276515237709701, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.04276515237709701
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.44999999999999984, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.44999999999999984
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15


RandomizedSearchCV(estimator=LGBMClassifier(), n_iter=5,
                   param_distributions={'boosting': ['gbdt', 'dart', 'rf'],
                                        'feature_fraction': array([0.1 , 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 ,
       0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31,
       0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42,
       0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53,
       0...
       101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
       127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
       140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
       153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
       166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178,
       179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191,
       192, 193, 194, 195, 196, 197, 198, 199])})

In [13]:
X_eval, y_eval = X_reweighted, y_reweighted
knn_bayes = LGBMClassifier(**gec.best_params_)
score_bayes = np.mean(cross_val_score(knn_bayes, X_eval, y_eval, cv=5))
knn_gs = LGBMClassifier(**gs.best_params_)
score_gs = np.mean(cross_val_score(knn_gs, X_eval, y_eval, cv=5))
knn_default = LGBMClassifier()
score_default = np.mean(cross_val_score(knn_default, X_eval, y_eval, cv=5))
print(f"bayesian: {score_bayes}, random search: {score_gs}, default: {score_default}")


[LightGBM] [Warning] feature_fraction is set=0.13, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.13
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=46, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=46
[LightGBM] [Warning] lambda_l2 is set=0.013702670330891934, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.013702670330891934
[LightGBM] [Warning] bagging_fraction is set=0.9500000000000001, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9500000000000001
[LightGBM] [Warning] lambda_l1 is set=0.06997231495784935, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.06997231495784935
[LightGBM] [Warning] feature_fraction is set=0.13, colsample_bytree=1.0 will be ignored. Current value: feature_frac

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.utils.extmath import cartesian
from sklearn.gaussian_process.kernels import RBF, ConstantKernel

gaussian = GaussianProcessRegressor(kernel=gec.kernel)
#gaussian.fit(result['uniform'][0], result['uniform'][0].dot(np.array([1.0, 3.0]))/500)
gaussian.fit(gec.gp_datas['gbdt-no_bagging']["inputs"], gec.gp_datas['gbdt-no_bagging']["output"])

In [ ]:
visualise_2D_gaussian_process(gaussian, (np.logspace(0.00, 1, 50) -1)/9, np.arange(10, 200, 1), np.arange(0.5, 1, 0.1))

In [ ]:
print(gec.best_params_)
knn_bayes = LGBMClassifier(**gec.best_params_)
score_bayes = np.mean(cross_val_score(knn_bayes, X, y, cv=5))
score_bayes

In [ ]:
knn_bayes = LGBMClassifier(**{'boosting': 'gbdt', 'lambda_l1': 0.46, 'num_leaves': 12})
score_bayes = np.mean(cross_val_score(knn_bayes, X, y, cv=5))
score_bayes